
# General information of the datascrapping process.

Here, we get vital data for the key airports we want to scrape data from:

location (coordinates, country, city), IATA code,

This then allows us, using the wikipedia API to find each airport's corresponding wikipedia link and wikipedia name.
For example JFK is encoded in wikipedia's internal database as John_F._Kennedy_International_Airport. This matches the last part of the url in the wikipedia page link for the airport:     
https://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport

For each time range:

The wikipedia page's raw text can be scrapped predictiably for a list of destinations from an airport. We will encode airport destinations using iata code.

We do this for each airport, generating a large csv file of airport-destination pairs for a particular time change.

We will look at 2 time ranges (now(as of June 4th 11am Eastern Time), before Jan 1, 2000 UTC 0)

### Sources :


list of top 1000 airports by traffic to scrape:

https://gettocenter.com/airports/top-100-airports-in-world/1000#google_vignette  


detailed airport database to cross reference:

https://www.partow.net/miscellaneous/airportdatabase/index.html#Downloads 

# Part 1: get basic data for the list of top 1000 airports 

In [35]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
# Suppress just SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:
# Example: load HTML from a URL or string
url = "https://gettocenter.com/airports/top-100-airports-in-world/1000"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table")  
rows = table.find_all("tr")

with open("./data/top_airports_basic_data.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for row in rows:
        # Extract all cells (td or th)
        cols = row.find_all(["td", "th"])
        # Write the row text content to CSV
        writer.writerow([col.get_text(strip=True) for col in cols])
    f.close()

after replacing blank strings "", read in again csv file and add column names

In [ ]:
data = pd.read_csv("./data/top_airports_basic_data.csv", names=["full_name", "iata", "city", "country", "estimated_pax"])
print(len(data))
data=data.dropna()
print(len(data))
data.head(n=11)

1087
989


,full_name,iata,city,country,estimated_pax
1.0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992"
2.0,Beijing Capital International Airport,PEK,Beijing,China,"95,786,442"
3.0,Dubai International Airport,DXB,Dubai,United Arab Emirates,"88,242,099"
4.0,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968"
5.0,O'Hare International Airport,ORD,Chicago,United States,"79,828,183"
6.0,Heathrow Airport,LHR,London,United Kingdom,"78,014,598"
7.0,Haneda Airport,HND,Tokyo,Japan,"76,476,251"
8.0,Hong Kong International Airport,HKG,Hong Kong,Hong Kong,"72,665,078"
9.0,Shanghai Pudong International Airport,PVG,Shanghai,China,"70,001,237"
10.0,Charles de Gaulle International Airport,CDG,Paris,France,"69,471,442"


we now have obtain basic information for the top airports the the world. save the pdf

In [ ]:
data.to_csv("./data/top_airports_basic_data.csv")

#  Part 2:
Get wikipedia urls for each iata code by using search api in wikipedia

In [21]:
#testing a function
def get_wikipedia_url_from_iata(iata_code):
    """Search Wikipedia using IATA code and return the best-matching article title."""
    search_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": f"{iata_code} airport",
        "format": "json"
    }

    response = requests.get(search_url, params=params)
    data = response.json()
    try:
        raw_name = data['query']['search'][0]['title'] 
        raw_name = raw_name.replace(" ", "_") #replace the raw name spaces with _

        return "https://en.wikipedia.org/wiki/"+raw_name #format for english wiki
    except (KeyError, IndexError):
        return None
    
get_wikipedia_url_from_iata("LGA")
get_wikipedia_url_from_iata("DFW")
get_wikipedia_url_from_iata("LAX")
get_wikipedia_url_from_iata("SFO")

'https://en.wikipedia.org/wiki/San_Francisco_International_Airport'

now, run for all airports

In [ ]:
data = pd.read_csv("./data/top_airports_basic_data.csv")

In [29]:
data["wiki_url"] =None #add column
for index, row in data.iterrows():
    try:
        print("current index:", index)
        iata = row["iata"]
        url = get_wikipedia_url_from_iata(iata)
        data["wiki_url"][index] = url
    except:
        data["wiki_url"][index] = None

current index: 0


C:\Users\maxji\AppData\Local\Temp\ipykernel_40900\1159726583.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["wiki_url"][index] = url


current index: 1


C:\Users\maxji\AppData\Local\Temp\ipykernel_40900\1159726583.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["wiki_url"][index] = url


current index: 2


...

C:\Users\maxji\AppData\Local\Temp\ipykernel_40900\1159726583.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["wiki_url"][index] = url


current index: 987


C:\Users\maxji\AppData\Local\Temp\ipykernel_40900\1159726583.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["wiki_url"][index] = url


current index: 988


C:\Users\maxji\AppData\Local\Temp\ipykernel_40900\1159726583.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["wiki_url"][index] = url


In [30]:
data.head()

,Unnamed: 0,full_name,iata,city,country,estimated_pax,wiki_url
0,1.0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992",https://en.wikipedia.org/wiki/Hartsfield–Jacks...
1,2.0,Beijing Capital International Airport,PEK,Beijing,China,"95,786,442",https://en.wikipedia.org/wiki/Beijing_Capital_...
2,3.0,Dubai International Airport,DXB,Dubai,United Arab Emirates,"88,242,099",https://en.wikipedia.org/wiki/Dubai_Internatio...
3,4.0,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968",https://en.wikipedia.org/wiki/Los_Angeles_Inte...
4,5.0,O'Hare International Airport,ORD,Chicago,United States,"79,828,183",https://en.wikipedia.org/wiki/O'Hare_Internati...


finding number of null entries in the wiki_url column

In [ ]:
print(f"Number of null entries: {data['wiki_url'].isnull().sum()}")

Number of null entries: 1


find the null row   

In [32]:
data[data['wiki_url'].isnull()]

,Unnamed: 0,full_name,iata,city,country,estimated_pax,wiki_url
465,466.0,Victoria International Airport,YYJ,Victoria,Canada,"1,934,842",None


add the missing url

In [33]:
data["wiki_url"][465] = "https://en.wikipedia.org/wiki/Victoria_International_Airport"
print(f"Number of null entries: {data['wiki_url'].isnull().sum()}")

Number of null entries: 0


C:\Users\maxji\AppData\Local\Temp\ipykernel_40900\550697600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["wiki_url"][465] = "https://en.wikipedia.org/wiki/Victoria_International_Airport"


saving the updated data

In [34]:
data.to_csv("./data/top_airports_basic_data.csv")